In [39]:
from joblib import dump, load
from pyproj import Transformer
import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np

x1,y1 = 6584000,540500
transformer = Transformer.from_crs("epsg:3301", "epsg:4326")# from lambert est97 coords nickname to wsg84 nickname

transformer.transform(x1,y1)

df = pd.read_csv("../../data/locations_in_use.csv")
df = df.apply(pd.to_numeric, errors='ignore', downcast='integer')
kohad = ['KOHT_APTEEK', 'KOHT_AVALIK_KOHT', 'KOHT_BUROO_KONTOR', 'KOHT_BUSSIJAAM_BUSSITERMINAAL', 'KOHT_BUSSIPEATUS', 'KOHT_EHITUS', 'KOHT_HOTELL_MOTELL', 'KOHT_KASIINO', 'KOHT_KAUBAMAJA', 'KOHT_KAUPLUS', 'KOHT_KIOSK', 'KOHT_KULTUURIASUTUS', 'KOHT_LASTEASUTUS_TERRITOORIUM', 'KOHT_MUU KOHT', 'KOHT_MUU RUUM', 'KOHT_OOKLUBI_DISKOTEEK', 'KOHT_OPPEASUTUS_SELLE_TERRITOORIUM', 'KOHT_OUEALATEE', 'KOHT_OUEALA_LAHTINE_HOOV', 'KOHT_PANGAAUTOMAAT', 'KOHT_PARK', 'KOHT_PARKLA', 'KOHT_RAND', 'KOHT_RAUDTEEJAAM', 'KOHT_RAVIASUTUS', 'KOHT_REISILAEV', 'KOHT_RIIDEHOID', 'KOHT_SADAM', 'KOHT_SOOGIKOHT', 'KOHT_SPORDIRAJATIS', 'KOHT_TAKSO', 'KOHT_TANAV_VALJAK', 'KOHT_TANKLA', 'KOHT_TEENINDUSETTEVOTE', 'KOHT_TREPIKODA_KORIDOR', 'KOHT_TURG', 'KOHT_UHISSOIDUK']
for k in kohad:
    df[k] = 0
df.KOHT_AVALIK_KOHT = 1
df.KOHT_BUSSIJAAM_BUSSITERMINAAL = 1
df.KOHT_BUSSIPEATUS = 1
df["ToimNadalapaev_Esmaspäev"] = 0
df["ToimNadalapaev_Kolmapäev"] = 0
df["ToimNadalapaev_Laupäev  "] = 0
df["ToimNadalapaev_Neljapäev"] = 1
df["ToimNadalapaev_Pühapäev"] = 0
df["ToimNadalapaev_Reede     "] = 0
df["ToimNadalapaev_Teisipäev"] = 0
df["Aasta"] = 2021
df["Kuu"] = 11
df["Päev"] = 4
df["Tund"] = 20
df.head()
y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)
y_valid.head()


,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS
0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [40]:
rfc = load("../models/rfc_firstiter.joblib")
print("loaded")

loaded


In [41]:
predicted_proba = rfc.predict_proba(df)


In [42]:
df_proba = pd.DataFrame(np.array([x.transpose()[1] for x in predicted_proba]).transpose(), columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud
for i in df_proba.columns:
    print(sum(df_proba[i]))

827.246666666666
130.08000000000078
288.27999999999975
13.469999999999796
28.04666666666697
82.17000000000034
0.04
12.709999999999846
312.53
155.0033333333343
24.010000000000385
4.2899999999999645
394.2599999999998
550.1399999999996


In [29]:
df_proba["Lest_Y"] = df.Lest_Y
df_proba["Lest_X"] = df.Lest_X

Index(['LIIK_AVALIKU_KORRA_RIKKUMINE', 'LIIK_JALGRATTA_MOPEEDI_VARGUS',
       'LIIK_KEHALINE_VAARKOHTLEMINE', 'LIIK_KELMUS', 'LIIK_KONFLIKT/OMAVOLI',
       'LIIK_KOOLIVAGIVALD', 'LIIK_LIIKLUSRIKKUMINE',
       'LIIK_MOOTORSOIDUKI_VARGUS', 'LIIK_MUU', 'LIIK_PISIVARGUS',
       'LIIK_ROOVIMINE', 'LIIK_TULEKAHJU', 'LIIK_VANDALISM', 'LIIK_VARGUS',
       'Lest_Y', 'Lest_X'],
      dtype='object')

In [31]:
fastmap = folium.Map([58.87, 25.49], zoom_start=7)

dfmatrix = df_proba[['Lest_X', 'Lest_Y']].values
dfmatrix = np.array([transformer.transform(row[0]+250,row[1]+250) for row in dfmatrix])     # see +250 on sest ainult esimene vahemikust, vahemiku suurus 500
fastmap.add_child(plugins.HeatMap(dfmatrix, radius=5, weights=list(df_proba["LIIK_AVALIKU_KORRA_RIKKUMINE"].values)))

fastmap
